In [ ]:
!pip install requests langchain-community beautifulsoup4 langchain chromadb sentence-transformers together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.1 MB/s eta 0:00

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import requests
import chromadb
from chromadb import Client
import pickle
from google.colab import drive
import numpy as np
import gensim.downloader
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from langchain.vectorstores import Chroma
from scipy.spatial.distance import cosine
from together import Together
from transformers import GPT2Tokenizer
import firebase_admin
from firebase_admin import credentials, firestore
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client=chromadb.PersistentClient(path="/content/drive/MyDrive/test")

In [ ]:
client.count_collections()

1

In [ ]:
rag=client.get_collection(name="rag_with_HF")

In [ ]:
rag.count()

965

# **Query and Retreival**

Embedding model for embedding the query

In [ ]:
# 1. Hugging Face Embedding
hf_embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

<ipython-input-18-98516c81e59e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


functions to calculate cosine similarity

In [ ]:
def calculate_cosine_similarity(query_embedding, document_embeddings):
    query_embedding = np.array(query_embedding)  # Convert to NumPy array

    # Ensure that the query and document embeddings are in the correct shape
    query_embedding = query_embedding.reshape(1, -1)  # Reshape to 2D array for cosine similarity
    return cosine_similarity(query_embedding, document_embeddings).flatten()  # Flatten to get a 1D array of similaritiess

similarity score with a threshold of 0.55.

Diversity of results returned by Maximal Marginal Relevance (MMR) is set to 0.25.

In [ ]:
def initial_query(collection, query_embedding, k=10):
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["documents", "embeddings"]
    )
    # Return list of results
    return results['documents'][0], results['embeddings'][0]

In [ ]:
def filter_by_similarity(documents, embeddings, query_embedding, threshold=0.7):

    document_embeddings = np.array(embeddings)  # Convert embeddings list to NumPy array
    if document_embeddings.ndim == 3:
        # Reshape to (10, 768)
        document_embeddings = document_embeddings.reshape(document_embeddings.shape[1], -1)

    # Calculate cosine similarities
    similarities = calculate_cosine_similarity(query_embedding, document_embeddings)

    # Filter documents based on threshold
    filtered_documents = []
    for doc, similarity, embedding in zip(documents, similarities, document_embeddings):
        if similarity >= threshold:
            filtered_documents.append((doc, similarity, embedding))  # Include embedding in the tuple

    return filtered_documents

In [ ]:
# Apply MMR for diversity
def apply_mmr(filtered_results, query_embedding, k, lambda_mult):
    selected_results = []

    # Convert query_embedding into a 1D array
    query_embedding = np.array(query_embedding).flatten() #shape(768,)

    for _ in range(k):
        if not filtered_results:  # Check if there are any results left to select
            break

        # Calculate the scores for each candidate in the filtered results
        candidate_scores = []
        for candidate in filtered_results:
            candidate_document = candidate[0]
            candidate_embedding = candidate[2]

            # Ensure the embedding is a NumPy array and flatten it
            candidate_embedding = np.array(candidate_embedding).flatten()
            # print(candidate_embedding.shape) #(768)(ddebugging)
            # Compute the mmr score based on the similarity to the query and the selected results
            score = lambda_mult * (1 - cosine(query_embedding, candidate_embedding)) - \
                    (1 - lambda_mult) * min(
                        [cosine(candidate_embedding, sel[2].flatten()) for sel in selected_results] or [1]
                    )
            candidate_scores.append((candidate, score))

        # Select the candidate with the highest score
        selected_candidate, _ = max(candidate_scores, key=lambda x: x[1])
        selected_results.append(selected_candidate)

        # Remove the selected candidate from the filtered results
        filtered_results.remove(selected_candidate)

    return selected_results

In [ ]:
# Parameters
k = 4
lambda_mult = 0.25
collection_name = "rag_with_HF"

In [ ]:
# query_text='hi'
query_text = "According to the ASEAN Human Rights Declaration, what is the primary responsibility of ASEAN Member States in promoting and protecting human rights?"
query_embedding = hf_embedding_function.embed_query(query_text)

In [ ]:
query_embedding = np.array(query_embedding)

if query_embedding.ndim > 1:
    query_embedding = query_embedding.flatten()

In [ ]:
def retrieve_final_results(client, collection_name, query_embedding, k, lambda_mult, similarity_threshold=0.7):
    # Retrieve the collection
    collection = client.get_collection(name=collection_name)

    # Execute the initial query
    initial_results_documents, initial_results_embeddings = initial_query(collection, query_embedding, k)
    # Filter the results based on similarity
    filtered_results = filter_by_similarity(initial_results_documents, initial_results_embeddings, query_embedding, similarity_threshold)
    # Apply MMR for diversity
    final_results = apply_mmr(filtered_results, query_embedding, k, lambda_mult)

    return final_results

In [ ]:
final_results = retrieve_final_results(client,collection_name, query_embedding, k, lambda_mult)

In [ ]:
tg_client = Together(api_key='b49d2dd93bb8afc46de91bb8c6d7d5827bedd471d414df36dd2cbcb7bb0aa786')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Define a template for handling unknown information
unknown_info_template = """
Response to the Query:

Thank you for your question! However, it appears that the information you've provided falls outside the scope of my training and knowledge base. Here are some suggestions on how you can approach this situation:

1. Contextual Clarification:
   - Please provide additional context or clarify your question. This might help me assist you better.

2. General Guidance:
   - While I may not have specific information on that topic, I can offer general advice or direct you to reliable sources.

3. Further Exploration:
   - Consider exploring reputable websites, academic journals, or subject-matter experts who specialize in the area you’re inquiring about.

4. Related Topics:
   - If you're interested, I can provide information or summaries related to similar topics that are within my knowledge base.

If you have other questions or topics you'd like to discuss, feel free to ask!
"""

In [ ]:
# Define the structured summarization prompt
summarization_prompt = """
Summarize the following document based on the question in a structured format:
Introduction about the topic question
Conclusion : Summary of the document
"""

In [ ]:
# Function to summarize the document using the Together client
def summarize_with_together_api(tg_client, question, document, max_tokens=4097):
    # Prepare prompt from the summarization template
    prompt = summarization_prompt.format(question=question, document=document)

    # Calculate the token limit for the document after accounting for the prompt overhead
    prompt_tokens = tokenizer.encode(prompt, return_tensors="pt")
    max_document_tokens = max_tokens - len(prompt_tokens[0]) - 1  # Reserve room for response tokens

    # Truncate document if necessary
    document_tokens = tokenizer.encode(document, return_tensors="pt")
    if len(document_tokens[0]) > max_document_tokens:
        truncated_document = tokenizer.decode(document_tokens[0][:max_document_tokens])
        prompt = summarization_prompt.format(question=question, document=truncated_document)

    try:
        # API call with the prompt
        response = tg_client.chat.completions.create(
            model="Gryphe/MythoMax-L2-13b-Lite",
            messages=[{"role": "user", "content": prompt}]
        )

        # Return the summary if response is valid
        if response and response.choices:
            return response.choices[0].message.content.strip()
        else:
            return "No summary available from the model."

    except Exception as e:
        return f"Error while calling the Together API: {str(e)}"

In [ ]:
def summarize_documents(tg_client, question, documents, max_tokens=4097):
    if not documents:
        # If documents list is empty
        return None

    # Concatenate all documents into one text block
    merged_document = " ".join(documents)

    # Summarize the concatenated document
    summary = summarize_with_together_api(tg_client, question, merged_document, max_tokens=max_tokens)

    # Return the summary or a placeholder if no summary was generated
    return summary if summary else "No summary available for the provided documents."

In [ ]:
# Function to display summarized results
def display_summarized_results(client, tg_client, collection_name, query_text, k, lambda_mult):

    # Prepare the query embedding
    query_embedding = hf_embedding_function.embed_query(query_text)
    query_embedding = np.array(query_embedding).flatten()  # Ensure 1D array

    # Retrieve results
    final_results = retrieve_final_results(client, collection_name, query_embedding, k, lambda_mult)

    # Generate summaries for the provided documents
    summaries = summarize_documents(tg_client, query_text, [doc[0] for doc in final_results],max_tokens=4097)

    # Check if any summaries were generated
    if not summaries:
        # Print the unknown information template if no documents are provided or no summaries are generated
        print(unknown_info_template)
    else:
       print(summaries)

In [ ]:
display_summarized_results(client, tg_client, collection_name, query_text, k, lambda_mult)

Introduction: The document discusses the importance of understanding the concept of "big data" and its impact on various industries. It emphasizes the need for professionals to develop skills in handling large datasets to stay competitive in their respective fields.

Conclusion: In summary, the document highlights the significance of big data in today's world and the necessity for professionals to acquire the necessary skills to manage and analyze large datasets. It also emphasizes the potential benefits that can be derived from leveraging big data in various industries.
